<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow2/Start/SequentialAPI/Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TensorFlow 2


## Coding tutorials

#### [1. Hello TensorFlow!](#coding_tutorial_1)

<a id='coding_tutorial_1'></a>
## Hello TensorFlow!

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

strategy = tf.distribute.MirroredStrategy()
batch_size = 64 * strategy.num_replicas_in_sync

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

ds_train = tf.data.Dataset.from_tensor_slices(tensors=(x_train, y_train))
ds_train = ds_train.map(map_func=lambda x, y: (
    tf.cast(x=x, dtype=tf.float32)/255., y))
ds_train = ds_train.map(map_func=lambda x, y: (
    x[..., tf.newaxis], y))
ds_train = ds_train.shuffle(buffer_size=len(ds_train))\
    .batch(batch_size=batch_size)
print("Train element spec:", ds_train.element_spec)

ds_test = tf.data.Dataset.from_tensor_slices(tensors=(x_test, y_test))
ds_test = ds_test.map(map_func=lambda x, y:
    (tf.cast(x=x, dtype=tf.float32)/255., y))
ds_test = ds_test.map(map_func=lambda x, y: (
    x[..., tf.newaxis], y))
ds_test = ds_test.batch(batch_size=batch_size)
print("Test element spec:", ds_test.element_spec)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

class MNISTClassifier(tf.keras.Model):
    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.features_extractor = FeaturesExtractor()
        self.classifier = Classifier(num_classes=num_classes)

    def call(self, inputs, training=False):
        h = self.features_extractor(inputs, training=training)
        return self.classifier(h)

class FeaturesExtractor(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(filters=8,
            kernel_size=(3, 3), padding="same", activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = tf.keras.layers.Flatten()
        
    def call(self, inputs, training=False):
        h = self.conv(inputs)
        h = self.batchnorm(h, training=training)
        h = tf.nn.relu(h)
        h = self.maxpool(h)
        return self.flatten(h)

class Classifier(tf.keras.layers.Layer):
    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(units=64,
            activation=tf.keras.activations.relu,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.dense2 = tf.keras.layers.Dense(units=64,
            activation=tf.keras.activations.relu,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.dense3 = tf.keras.layers.Dense(units=num_classes,
            activation=tf.keras.activations.softmax,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
    
    def call(self, inputs):
        h = self.dense1(inputs)
        h = self.dense2(h)
        return self.dense3(h)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

with strategy.scope():
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        reduction=tf.keras.losses.Reduction.NONE)

    @tf.function
    def compute_loss(y_true, y_pred, model_losses):
        per_example_loss = loss_fn(y_true=y_true, y_pred=y_pred)
        loss = tf.nn.compute_average_loss(per_example_loss)
        if model_losses:
            loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))
        return loss
    
    model = MNISTClassifier(num_classes=10)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

    train_loss = tf.keras.metrics.Mean(name="train_loss")
    train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="train_acc")
    
    test_loss = tf.keras.metrics.Mean(name="test_loss")
    test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="test_acc")

In [ ]:
@tf.function
def train_step(inputs):
    x_train, y_train = inputs
    
    with tf.GradientTape() as tape:
        y_pred = model(inputs=x_train, training=True)
        loss = compute_loss(y_train, y_pred, model.losses)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss_fn(y_true=y_train, y_pred=y_pred))
    train_acc.update_state(y_true=y_train, y_pred=y_pred)
    return loss

@tf.function
def distributed_train_step(ds_inputs):
    per_replica_losses = strategy.run(train_step, args=(ds_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

for epoach in range(2):
    total_loss = 0.0
    num_batches = 0
    for ds in ds_train:
        total_loss += distributed_train_step(ds)
        num_batches += 1
    train_loss = total_loss / num_batches
    print(train_loss)

In [ ]:
# Import Tensorflow
import tensorflow as tf
tf.keras.utils.set_random_seed(42)
# Check its version & devices
print(tf.__version__)
print(tf.config.list_physical_devices())

# Train a feedforward neural network for image classification
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(
    units=16, activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.Dense(
    units=10, activation=tf.keras.activations.softmax
))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
print("Training model...\n")
history = model.fit(
    x=x_train, y=y_train, validation_split=0.1,
    epochs=10, batch_size=32, verbose=0
)
print("Model trained successfully")

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

df_history = pd.DataFrame(data=history.history, index=history.epoch)
metrics = ["loss", "sparse_categorical_accuracy"]
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
for ax, metric in zip(axes, metrics):
    sns.lineplot(ax=ax, data=df_history, label="train",
        x=df_history.index+1, y=metric
    )
    try:
        sns.lineplot(ax=ax, data=df_history, label="valid",
            x=df_history.index+1, y=f"val_{metric}"
        )
    except Exception:
        pass
    ax.legend(loc="best")
    ax.set_xlabel(xlabel="epoch")
    ax.set_ylabel(ylabel=f"{metric}")
    ax.set_title(label=f"{metric} vs. epoch")
fig.tight_layout()
plt.show()